<a href="https://colab.research.google.com/github/cincinnatilibrary/collection-analysis/blob/master/misc/chpl_rest_api_sierra_bulk_item_level_holds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://ilsweb.cincinnatilibrary.org/img/CHPL_Brandmark_Primary.png" alt="CHPL" title="CHPL" width="250"/>

# REST API: Bulk Item-level Holds

For various collection-health purposes, this script can be used in conjunction with CollectionHQ to move groups of items from one location to another using the holds functionality.

The script expects a Excel workbook with 1 sheet having the following format:

| barcode       | branch pickup location	|
|-------------- |------------------------	|
| A000055325526 | an |
| A000057896540 | ba |
| A000067386443 | dp |
| A000071676870 | pr |

In this example, the script will place four item-level holds, and set the locations to the coorisponding pickup location--using their pickup location codes (Anderson, Blue Ash, Deer Park, Price Hill)

---

*Note*: The current 5-character pickup location codes can be found here:

https://github.com/cincinnatilibrary/collection-analysis/blob/master/misc/chpl_rest_api_sierra_pickup_locations.ipynb



In [ ]:
# <--- Click the play button to the left to get started

# this is the helper script that contains some needed functions
!wget --quiet https://raw.githubusercontent.com/cincinnatilibrary/collection-analysis/master/misc/chpl_helper.py --output-document=chpl_helper.py

# import the helper functions we just downloaded ...
import chpl_helper
from getpass import getpass
from google.colab import files
import pandas as pd
import requests
import json


# point this to the base URL for the Sierra REST API
base_url = 'https://classic.cincinnatilibrary.org:443/iii/sierra-api/v6/'

## 1. Enter REST-API Credentials

In order to be authorized to use the REST API, you must enter valid credientials.

When prompted, enter in the **`auth_string`** as your password:

In [ ]:
# client_key = getpass('Enter the client KEY value: ')
# client_secret = getpass('Enter the client SECRET value: ')
auth_string = getpass('Enter the auth_string value: ')

# test our credentials

headers = chpl_helper.set_access_headers(base_url, auth_string=auth_string)
r = requests.get(base_url + 'info/token', headers=headers, verify=True)
print('access token expires in: {} seconds'.format(r.json()['expiresIn']))

## 2. Upload Input File

This file will be processed to placed the holds.

When prompted, upload the input file:

In [ ]:
input_file = files.upload()

In [ ]:
# convert the input file to a Dataframe ... 
df = pd.read_excel( 
    [key for key in input_file.keys()][0], 
    converters={
        'barcode': str,        
    }
)
print(df.head(5))
print('...')
print('total rows in input: ', df.shape[0])

# construct the item query we'll use to convert barcodes to item record numbers (API endpoint URLs)
operands = [row['barcode'] for i, row in df.iterrows()]
item_query = {"queries": [{"target": {"record": {"type": "item"}, "field": {"tag": "b"}}, "expr": [{"op": "in", "operands": operands}]}]}

print(json.dumps(item_query))

headers = chpl_helper.set_access_headers(base_url, auth_string=auth_string)

print('\nsearching ...')

r = requests.post(
    base_url + 'items/query?offset=0&limit={}'.format(len(operands)), 
    headers=headers,
    data=json.dumps(item_query),
    verify=True
)

print("response json: ", r.json())

print('...')

print('total rows of items from search: {}'.format(r.json()['total']) )

In [ ]:
for i, entry in enumerate(r.json()['entries']):
    print(i, entry['link'])
    r_entry = requests.get(
        url=entry['link'] + '?fields=id,location,status,barcode,bibIds',
        headers=headers,
        verify=True
    )
    print(r_entry.json())
    
    # extract the item record number
    item_record_num = r_entry.json()['id']
        
    #extract the barcode
    search_barcode = str(r_entry.json()['barcode'])
    
    # NOTE: for the time being, we'll just use this patron record for testing
    # https://classic.cincinnatilibrary.org/iii/sierra-api/v6/patrons/2509457
    patron_record_num = '2509457'

    # match the barcode to get the pickup location code in the input file / Dataframe 
    pickupLocation = df[df.barcode == search_barcode]['branch pickup location'].values[0]
    # patron_record_num = df[df.barcode == search_barcode].branch_account.values[0].strip()[1:-1]
    
    print('placing hold...')

    print(f'item_record_num:\t{item_record_num}\nsearch_barcode:\t\t{search_barcode}\npatron_record_num:\t{patron_record_num}')

    # patron_record_num = df[df.barcode == search_barcode].branch_account.values[0].strip()[1:-1]
    
    # place the hold ...
    
    body_data = {
        "recordType": "i",
        "recordNumber": int(item_record_num),
        "pickupLocation": pickupLocation,
        # "neededBy": "",
        # "numberOfCopies": 1,
        "note": "chpl-bulk-hold-script"
    }

    print(base_url + f'patrons/{patron_record_num}/holds/requests', json.dumps(body_data))
    
    r_hold = requests.post(
        url=base_url + f'patrons/{patron_record_num}/holds/requests',
        headers=headers,
        data=json.dumps(body_data),
        verify=True
    )

    # print("response json: ", r_hold.json())
    print('response:\n', r_hold.status_code, '\n', r_hold.content, sep='')
    
    print('---')

In [ ]:
#  delete /v6/patrons/{id}/holds 

# cancel all the holds for this patron account
r_cancel = requests.delete(
        url=base_url + f'patrons/{patron_record_num}/holds',
        headers=headers,
        verify=True
)

print('response:\n', r_cancel.status_code, '\n', r_cancel.content, sep='')